In [100]:
import pandas as pd
import os
from imdb.imdb import IMDb

In [101]:
imdb_ratings = pd.read_csv(f'raw/title.ratings.tsv', sep='\t', na_values="\\N")
imdb_titles = pd.read_csv(f'raw/title.basics.tsv', sep='\t', na_values="\\N")

imdb_merged = pd.merge(
    imdb_titles[imdb_titles['titleType'] == 'movie'],
    imdb_ratings,
    on='tconst',
).fillna({'averageRating': -1, 'numVotes': -1, 'runtimeMinutes': -1, 'genres': ''})

C:\Users\danpf\AppData\Local\Temp\ipykernel_21968\2287869136.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  imdb_titles = pd.read_csv(f'raw/title.basics.tsv', sep='\t', na_values="\\N")


In [102]:
imdb_merged['startYear'] = imdb_merged['startYear'].astype(str)
imdb_merged['startYear'] = imdb_merged['startYear'].str[:4]

In [103]:
imdb_merged.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0000009,movie,Miss Jerry,Miss Jerry,0.0,1894,NaN,45.0,Romance,5.4,218
1,tt0000147,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0.0,1897,NaN,100.0,"Documentary,News,Sport",5.3,549
2,tt0000502,movie,Bohemios,Bohemios,0.0,1905,NaN,100.0,,3.8,20
3,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0.0,1906,NaN,70.0,"Action,Adventure,Biography",6.0,969
4,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0.0,1907,NaN,90.0,Drama,5.6,30


In [ ]:
reviews_df = pd.read_csv(f"raw/rotten_tomatoes_movie_reviews.csv")
movies_df = pd.read_csv(f"raw/rotten_tomatoes_movies.csv")

tomatoes_merged = pd.merge(
    movies_df[['id', 'title', 'audienceScore', 'tomatoMeter', 'releaseDateTheaters']],
    reviews_df[['id', 'creationDate', 'reviewState']],
    on='id'
).fillna({'tomatoMeter': -1, 'audienceScore': -1, 'releaseDateTheaters': -1})

In [105]:
tomatoes_merged['releaseYear'] = tomatoes_merged['releaseDateTheaters'].str[:4]

In [106]:
tomatoes_merged.head()

,id,title,audienceScore,tomatoMeter,releaseDateTheaters,genre,creationDate,reviewState,releaseYear
0,love_lies,"Love, Lies",43.0,NaN,NaN,Drama,2020-10-31,fresh,NaN
1,love_lies,"Love, Lies",43.0,NaN,NaN,Drama,2016-06-15,fresh,NaN
2,adrift_2018,Adrift,65.0,69.0,2018-06-01,"Adventure, Drama, Romance",2022-05-29,fresh,2018
3,adrift_2018,Adrift,65.0,69.0,2018-06-01,"Adventure, Drama, Romance",2022-02-11,fresh,2018
4,adrift_2018,Adrift,65.0,69.0,2018-06-01,"Adventure, Drama, Romance",2021-08-21,rotten,2018


In [107]:
filtered_df = pd.merge(
    imdb_merged,
    tomatoes_merged,
    left_on=['primaryTitle', 'startYear'],
    right_on=['title', 'releaseYear'],
    how='inner'
)

In [108]:
filtered_df = filtered_df.drop(columns=['primaryTitle', 'releaseYear', 'titleType', 'isAdult', 'endYear', 'id', 'originalTitle'])

In [109]:
filtered_df.head()

,tconst,startYear,runtimeMinutes,genres,averageRating,numVotes,title,audienceScore,tomatoMeter,releaseDateTheaters,genre,creationDate,reviewState
0,tt0003657,1914,70.0,Western,6.5,264,The Bargain,NaN,NaN,1914-12-03,Western,2021-06-29,fresh
1,tt0004099,1914,59.0,"Adventure,Comedy,Family",5.3,547,"His Majesty, the Scarecrow of Oz",53.0,NaN,1914-10-14,Fantasy,2005-02-01,fresh
2,tt0004223,1914,105.0,"Action,Adventure,Biography",6.1,78,The Life of General Villa,NaN,80.0,1914-05-09,"Action, Adventure, Drama",2020-07-28,fresh
3,tt0004223,1914,105.0,"Action,Adventure,Biography",6.1,78,The Life of General Villa,NaN,80.0,1914-05-09,"Action, Adventure, Drama",2020-07-22,fresh
4,tt0004223,1914,105.0,"Action,Adventure,Biography",6.1,78,The Life of General Villa,NaN,80.0,1914-05-09,"Action, Adventure, Drama",2020-07-18,rotten


In [110]:
filtered_df.shape

(867522, 13)

In [111]:
filtered_df.to_csv('raw/movies.csv')

In [112]:
len(set(list(zip(filtered_df.title, filtered_df.startYear))))

16127

In [113]:
interstellar = filtered_df[filtered_df['title'] == 'Interstellar']

In [114]:
list(zip(interstellar.creationDate, interstellar.reviewState))

[('2023-04-04', 'fresh'),
 ('2023-02-27', 'fresh'),
 ('2023-01-24', 'rotten'),
 ('2022-08-22', 'fresh'),
 ('2022-06-30', 'fresh'),
 ('2022-05-27', 'fresh'),
 ('2022-02-11', 'fresh'),
 ('2021-10-09', 'fresh'),
 ('2021-08-10', 'fresh'),
 ('2021-02-02', 'fresh'),
 ('2020-12-04', 'rotten'),
 ('2020-09-20', 'fresh'),
 ('2020-09-12', 'fresh'),
 ('2020-09-03', 'fresh'),
 ('2020-08-26', 'rotten'),
 ('2020-07-16', 'fresh'),
 ('2020-07-07', 'fresh'),
 ('2020-06-30', 'fresh'),
 ('2020-06-26', 'fresh'),
 ('2020-05-27', 'fresh'),
 ('2020-03-26', 'fresh'),
 ('2020-03-11', 'fresh'),
 ('2020-01-09', 'rotten'),
 ('2019-11-27', 'fresh'),
 ('2019-11-20', 'rotten'),
 ('2019-10-27', 'fresh'),
 ('2019-09-06', 'fresh'),
 ('2019-08-07', 'fresh'),
 ('2019-08-02', 'fresh'),
 ('2019-07-30', 'rotten'),
 ('2019-07-16', 'fresh'),
 ('2019-07-05', 'fresh'),
 ('2019-05-31', 'fresh'),
 ('2019-04-30', 'fresh'),
 ('2019-04-03', 'fresh'),
 ('2019-02-26', 'fresh'),
 ('2019-02-02', 'rotten'),
 ('2019-01-25', 'fresh'),
 ('20